In [16]:
import pandas as pd
from pyomo.environ import *
from IPython import display
from idaes.core.io.tests.test_common import _demo_model as demo_model
from io import StringIO
from ipydatagrid import DataGrid, TextRenderer, Expr

In [5]:
def bracket_split(s):
    p = s.split(".")
    q = []
    prev_x = None
    for x in p:
        if x[-1] == "]":
            q.append(prev_x + "." + x)
            prev_x = None
        else:
            if prev_x is not None:
                q.append(prev_x)
            prev_x = x
    if prev_x is not None:
        q.append(prev_x)
    return q

In [11]:
def var_table(m):
    max_name = 0
    for c in m.component_objects():
        if c.is_variable_type():
            max_name = max(max_name, len(bracket_split(c.name)))
    
    fields = {}
    for i in range(max_name):
        fields[f"name_{i + 1}"] = []
    fields.update({"desc": [], "input": [], "output": [], "value": [], "lower": [], "upper": [], "units": []})
    df = pd.DataFrame(fields)
    
    for c in m.component_objects():
        if c.is_variable_type():
            names = bracket_split(c.name)
            for i in range(max_name - len(names)):
                names.append("")
            row = names + ["", False, False, str(c.value), str(c.lb), str(c.ub), str(c.get_units())]
            df.loc[len(df)] = row
    return df

In [7]:
m = demo_model()

In [12]:
df = var_table(m.fs)

In [33]:
def bool_circle(cell):
    return "\uf111" if cell.value > 0 else " "
def bool_circle_color(cell):
    return "#2fbd34" if cell.value > 0 else "#b82538"
def text_renderer(field, func):
    styles = {
        "font": "bold 14px fontawesome",
        "text_color": Expr(bool_circle_color),
        "horizontal_alignment": "center",
    }
    return TextRenderer(text_value=Expr(func), **styles)
class BoolHandler:
    "One-click change to a boolean value"
    bool_columns = {"input", "output"}
    def __init__(self, grid):
        self.grid = grid
    def cell_clicked(self, cell):
        if cell["column"] not in self.bool_columns:
            return
        new_value = not cell["cell_value"]
        self.grid.set_cell_value_by_index(cell["column"], cell["row"], new_value)
        
renderers = {f: text_renderer(f, bool_circle) for f in ["input", "output"]}
grid = DataGrid(df, editable=True, renderers=renderers)
bhandler = BoolHandler(grid)
grid.on_cell_click(bhandler.cell_clicked)
grid

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…